# OCNC DLC Tutorial

Please make sure you are selecting the kernel that has your DLC installation!

In [ ]:
import deeplabcut as dlc

For some sanity checks if the working directory with the video of your choice is in the right location.
So, please change ```/Users/saffirayantjon/Documents/DLC/OCNC_DLC_Tutorial/``` to your local path.

- `pwd` prints your current local path 
- `ls` prints the directory information specified

In [ ]:
%%sh
pwd
ls /Users/saffirayantjon/Documents/DLC/OCNC_DLC_Tutorial/

## Create a project

For organisational sanity, I prefer to declare more global variables in the beginning of the file. This way when I export the job to a potential computing cluster, I can easily change the location etc. 

In [ ]:
""" ==== CREATE PROJECT ==== """

# -- edit this part to match your directory --
# -- input: str 
PROJECT_NAME: str = ""
EXPERIMENTER: str = ""
VIDEO_DIR: str = "" 
# WORKING_DIR: str = "models/"

# NOTE: the VIDEO_DIR is a lst input -- you can use 
dlc.create_new_project(PROJECT_NAME, EXPERIMENTER, [VIDEO_DIR+"M_190124_110324_12_60fps.avi"]
)

Check if it is creating the folders it promised you :)

In [ ]:
%%sh
ls 

### Check The project details & edit accordingly

In [ ]:
CONFIG_PATH: str = "/Users/saffirayantjon/Documents/DLC/OCNC_DLC_Tutorial/OCNC-Saffira-2025-06-27/config.yaml"



DISCLAIMER: I have anxiety and don't trust my fat fingers. For sane people, it is completely OK to open the file and just edit the config file manually using your text editor of choice. 

Please note:
- `numframes2pick`: Number of frames to label per video.
- `skeleton`: The body part label connections.
- `bodyparts`: The labels.

In [ ]:
import yaml

project_details = yaml.safe_load(open(CONFIG_PATH))
project_details

In [ ]:
# -- check number of frames to be extracted
project_details["numframes2pick"] = 20
project_details["numframes2pick"]

In [ ]:
# NOTE: Choose the method that allows you to select the frames that capture all the behaviour you are interested in :)
dlc.extract_frames(CONFIG_PATH, 
                   mode='automatic',
                   userfeedback=False, 
                   crop=False
)

Check if it actually did the thing it promised :) (folder in labeled-frames)
You should see something like: `img0015.png` 20 times a different frame.

In [ ]:
%%sh
ls 
open 

In [ ]:
# --- sanity check -- what are the current labels
bodyparts_details = project_details["bodyparts"]
print("Current bodyparts:")
print("--", bodyparts_details)

project_details["bodyparts"] =  ["LFP", # left front paw
                                 "RFP", # right front paw
                                 "LHP", # left hind paw
                                 "RHP", # right hind paw
                                 "TB", # tail base
                                 "T0", # tail segment 0
                                 "N" # nose
]

In [ ]:
# --- sanity check -- what are the current skeleton config
skeleton_details = project_details["skeleton"]
print("Current skeleton:")
print("--", skeleton_details)

project_details["skeleton"] = [[]]*4
project_details["skeleton"][0] = ["LF", "LHP"]
project_details["skeleton"][1] = ["RFP", "RHP"]
project_details["skeleton"][2] = ["N", "TB"]
project_details["skeleton"][3] = ["TB", "T0"]

In [ ]:
# -- write edit to the yaml file
yaml.safe_dump(
        project_details, open(CONFIG_PATH, "w"), sort_keys=False
)

Final check before labeling frames ! 

In [ ]:
project_details

In [ ]:
dlc.label_frames(CONFIG_PATH)

In [ ]:
dlc.check_labels(CONFIG_PATH)

## The steps below only if you have GPU engagement 
- Otherwise move onto Google Colab: https://colab.research.google.com/github/DeepLabCut/DeepLabCut/blob/master/examples/COLAB/COLAB_YOURDATA_TrainNetwork_VideoAnalysis.ipynb
    -   Copy to drive button! 

In [ ]:
CONFIG_PATH: str = "/Users/saffirayantjon/Documents/DLC/OCNC_DLC_Tutorial/OCNC-Saffira-2025-06-26/config.yaml"
dlc.create_training_dataset(CONFIG_PATH)

In [ ]:
dlc.train_network(CONFIG_PATH,shuffle=1,trainingsetindex=0,
                #   device="cuda:0",
            max_snapshots_to_keep=5,displayiters=100,save_epochs=5,epochs=200
)

In [ ]:
%matplotlib inline
dlc.evaluate_network(CONFIG_PATH,plotting=True, Shuffles=[1]
)

In [ ]:
%%sh
open 

This is an example evaluation result:

<img src="fig/Training-M_190124_110324_12_60fps-img0015.png" alt="original image" align="middle" width="720"/>

Compare the "+" and "keypoint".

In [ ]:
VIDEO_DIR:str = "/Users/saffirayantjon/Documents/DLC/OCNC_DLC_Tutorial/video/" 
dlc.analyze_videos(CONFIG_PATH, videos=[VIDEO_DIR+"M_190124_110324_12_30fps_30s.mp4"], videotype='mp4', 
                   shuffle=1, trainingsetindex=0, gputouse=None, save_as_csv=False,
                   destfolder=VIDEO_DIR, batchsize=1, cropping=None,
                   dynamic=(False, 0.5, 10)
)

In [ ]:
dlc.filterpredictions(CONFIG_PATH, video=VIDEO_DIR+"M_190124_110324_12_30fps_30s.mp4", 
                      videotype=".mp4",filtertype="median",shuffle=1,destfolder=VIDEO_DIR
)

In [ ]:
dlc.create_labeled_video(CONFIG_PATH, videos=[VIDEO_DIR+"M_190124_110324_12_30fps_30s.mp4"], videotype='.mp4',
                         shuffle=1,trainingsetindex=0, filtered=True,
                         save_frames=False, Frames2plot=None,
                         displayedbodyparts='all',
                         outputframerate=30, destfolder=VIDEO_DIR,
                         draw_skeleton=False, trailpoints=3, alphavalue=.7,displaycropped=False
)